In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

In [3]:
appearances = pd.read_csv("data/appearances.csv")
games = pd.read_csv("data/games.csv")
players = pd.read_csv("data/players.csv")

In [4]:
appearances

,appearance_id,game_id,player_id,player_club_id,date,player_pretty_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2483937_52453,2483937,52453,28095,2014-08-08,Haris Handzic,RU1,0,0,0,0,90
1,2479929_67064,2479929,67064,28095,2014-08-03,Felicio Brown Forbes,RU1,0,0,0,0,90
2,2483937_67064,2483937,67064,28095,2014-08-08,Felicio Brown Forbes,RU1,0,0,0,0,90
3,2484582_67064,2484582,67064,28095,2014-08-13,Felicio Brown Forbes,RU1,0,0,0,0,55
4,2485965_67064,2485965,67064,28095,2014-08-16,Felicio Brown Forbes,RU1,0,0,0,0,90
...,...,...,...,...,...,...,...,...,...,...,...,...
1106705,3839700_261546,3839700,261546,306,2022-09-10,Joost Van Aken,NL1,0,0,0,0,90
1106706,3839714_261546,3839714,261546,306,2022-09-18,Joost Van Aken,NL1,0,0,0,0,90
1106707,3852611_261546,3852611,261546,306,2022-11-05,Joost Van Aken,NL1,0,0,0,0,8
1106708,3852626_261546,3852626,261546,306,2022-11-13,Joost Van Aken,NL1,0,0,0,0,1


In [5]:
games

,game_id,competition_id,competition_type,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,...,home_club_position,away_club_position,club_home_pretty_name,club_away_pretty_name,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,url
0,2219794,BESC,other,2011,Final,2012-07-22,58,498,3,2,...,-1,-1,Rsc Anderlecht,Ksc Lokeren,John van den Brom,Peter Maes,Constant Vanden Stock Stadion,14485,Laurent Colemonts,https://www.transfermarkt.co.uk/spielbericht/i...
1,2244388,SUC,other,2012,final 1st leg,2012-08-22,131,418,3,2,...,-1,-1,Fc Barcelona,Real Madrid,Tito Vilanova,José Mourinho,Spotify Camp Nou,91728,Carlos Clos Gómez,https://www.transfermarkt.co.uk/spielbericht/i...
2,2211607,NLSC,other,2012,Final,2012-08-05,383,610,4,2,...,-1,-1,Psv Eindhoven,Ajax Amsterdam,Dick Advocaat,Frank de Boer,Johan Cruijff ArenA,50000,Björn Kuipers,https://www.transfermarkt.co.uk/spielbericht/i...
3,2252846,POSU,other,2012,Final,2012-08-11,720,2990,1,0,...,-1,-1,Fc Porto,Academica Coimbra,Vítor Pereira,Pedro Emanuel,Estádio do Dragão,26000,Olegário Benquerença,https://www.transfermarkt.co.uk/spielbericht/i...
4,2229332,DFL,other,2012,Final,2012-08-12,27,16,2,1,...,-1,-1,Fc Bayern Munchen,Borussia Dortmund,Jupp Heynckes,Jürgen Klopp,Allianz Arena,69000,Michael Weiner,https://www.transfermarkt.co.uk/spielbericht/i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59191,3888988,GR1,domestic_league,2022,1. Matchday,2022-08-20,3060,653,3,1,...,3,11,Atromitos Athen,Ofi Kreta,Chris Coleman,Nikolaos Nioplias,Stadio Peristeriou,715,Aristotelis Diamantopoulos,https://www.transfermarkt.co.uk/atromitos-athe...
59192,3889011,GR1,domestic_league,2022,4. Matchday,2022-09-10,653,6418,1,2,...,11,6,Ofi Kreta,Panetolikos Gfs,Nikolaos Nioplias,Giannis Anastasiou,Gipedo Theodoros Vardinogiannis,3014,Ioannis Papadopoulos,https://www.transfermarkt.co.uk/ofi-crete-fc_p...
59193,3889048,GR1,domestic_league,2022,9. Matchday,2022-10-22,6418,683,0,2,...,7,4,Panetolikos Gfs,Olympiakos Piraus,Giannis Anastasiou,Michel,Gipedo Panetolikou,3288,Andreas Gamaris,https://www.transfermarkt.co.uk/panetolikos-gf...
59194,3888990,GR1,domestic_league,2022,1. Matchday,2022-08-21,683,2671,2,0,...,4,12,Olympiakos Piraus,Pas Giannina,Carlos Corberán,Athanasios Staikos,Stadio Georgios Karaiskakis,9523,Vangelis Manouchos,https://www.transfermarkt.co.uk/olympiacos-pir...


In [6]:
players

,player_id,pretty_name,club_id,club_pretty_name,current_club_id,country_of_citizenship,country_of_birth,city_of_birth,date_of_birth,position,...,height_in_cm,market_value_in_gbp,highest_market_value_in_gbp,agent_name,contract_expiration_date,domestic_competition_id,club_name,image_url,last_season,url
0,254016,Arthur Delalande,855,Ea Guingamp,855,France,France,Saint-Malo,1992-05-18,Midfield,...,186,NaN,90000.0,NaN,NaN,FR1,ea-guingamp,NaN,2013,https://www.transfermarkt.co.uk/arthur-delalan...
1,51053,Daniel Davari,23,Eintracht Braunschweig,23,Iran,Germany,Gießen,1988-01-06,Goalkeeper,...,192,135000.0,1130000.0,NG360,2022-06-30,L1,eintracht-braunschweig,NaN,2013,https://www.transfermarkt.co.uk/daniel-davari/...
2,31451,Torsten Oehrl,23,Eintracht Braunschweig,23,Germany,Germany,Lichtenfels,1986-01-07,Attack,...,192,NaN,1130000.0,NaN,NaN,L1,eintracht-braunschweig,NaN,2013,https://www.transfermarkt.co.uk/torsten-oehrl/...
3,44622,Vladimir Kisenkov,3691,Tom Tomsk,3691,Russia,UdSSR,Kaluga,1981-10-08,Defender,...,182,NaN,720000.0,NaN,NaN,RU1,tom-tomsk,NaN,2013,https://www.transfermarkt.co.uk/vladimir-kisen...
4,30802,Oscar Diaz,3302,Ud Almeria,3302,Spain,Spain,Madrid,1984-04-24,Attack,...,183,NaN,1080000.0,Pedro Bravo - Consulting,2022-06-30,ES1,ud-almeria,NaN,2013,https://www.transfermarkt.co.uk/oscar-diaz/pro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25619,256718,Sven Van Beek,306,Sc Heerenveen,306,Netherlands,Netherlands,Gouda,1994-07-28,Defender,...,189,1000000.0,5500000.0,H2 Agency,2025-06-30,NL1,sc-heerenveen,https://img.a.transfermarkt.technology/portrai...,2022,https://www.transfermarkt.co.uk/sven-van-beek/...
25620,646704,Maxim Dekker,1090,Az Alkmaar,1090,Netherlands,Netherlands,Rijsenhout,2004-04-21,Defender,...,0,1500000.0,1500000.0,NaN,2026-06-30,NL1,az-alkmaar,https://img.a.transfermarkt.technology/portrai...,2022,https://www.transfermarkt.co.uk/maxim-dekker/p...
25621,454628,Alex Timossi Andersson,306,Sc Heerenveen,306,Sweden,Sweden,Helsingborg,2001-01-19,Attack,...,170,1200000.0,1500000.0,MD Management,2026-06-30,NL1,sc-heerenveen,https://img.a.transfermarkt.technology/portrai...,2022,https://www.transfermarkt.co.uk/alex-timossi-a...
25622,462988,Danil Lipovoy,2696,Krylya Sovetov Samara,2696,Russia,Russia,Kirov,1999-09-22,Midfield,...,177,300000.0,350000.0,NaN,2024-06-30,RU1,krylya-sovetov-samara,https://img.a.transfermarkt.technology/portrai...,2022,https://www.transfermarkt.co.uk/danil-lipovoy/...
